# Single Cycle CPU Assignment
## This Jupyter notebook will run the design on FPGA and display the maximum frequency of the design

In [ ]:
from pynq import Overlay, Clocks

In [ ]:
# Function to write instructions to registers
def write(idata):
    f = open(idata, "r")
    for count, line in enumerate(f.readlines()):
        cpu_ip.write(int(count*4),int(line, 16))
    

# Function to read RISCV 32 Registers
def read():
    out = []
    for i in range(32,64):
        out.append(cpu_ip.read(int(i*4)))
    return out


In [ ]:
clocks = [50, 100, 125, 142, 200]
for i in range(0,6):
    print("Testing "+"idata"+str(i+1)+".txt")
    f = open("expout"+str(i+1)+".txt", "r")        # Expected output file
    out_crct = []
    for line in f.readlines():
        out_crct.append(int(line))
    for clock in clocks:
        ol_cpu = Overlay('overlay/harness_axi.bit')    # Programming the bitstream onto the FPGA
        cpu_ip = ol_cpu.harness_axi_ip_v1_0_0          # RISCV CPU Verilog IP
        Clocks.fclk1_mhz = clock                       # Set clock frequency in MHz
        cpu_ip.write(int(64*4),1)                      # Assign reset to 1
        write("idata"+str(i+1)+".txt")                 # Write instructions
        cpu_ip.write(int(64*4),1)                      # Assign reset to 1
        cpu_ip.write(int(64*4),0)                      # Assign reset to 0
        out = read()                                   # Read registers
        
        
        # Increase the clock frequency untill timing violations occur
        if out==out_crct:                              # Check if expected and obtained register values
            print("\t", "Frequency at", clock, "MHz: Passed")
            clock += 50
        else:
            print("\t", "Frequency at", clock, "MHz: Not passed")
#             Use below lines for debug
#             print("Expected result: ", out_crct)
#             print("Obtained result: ", out)


We can observe that most of the programs work fine upto 100MHz frequency